# Kotlin programming

This function should return a list containing only the numbers that equal or exceed the threshold.

In [31]:
fun pruneList(list: List<Int>, threshold: Int): List<Int> = list.filter { it >= threshold }

require(pruneList(listOf(1, 2, 3, 4, 5, 6), 5) == listOf(5, 6))

This function should find the smallest element in the list:

In [19]:
fun smallest(list: List<Int>): Int = list.min()

require(smallest(listOf(5, 3, 2, 4)) == 2)

The `stats` function should return more general stats about the list:

* Min value
* Max value
* Sum of all values
* Mean of all values

We have a `Stats` interface which captures all of these:

In [32]:
import java.math.BigInteger
import kotlin.collections.fold

interface Stats {
    val min: Int
    val max: Int
    val sum: Long
    val mean: Double
}

data class StatsImpl(
    override val min: Int,
    override val max: Int,
    override val sum: Long,
    override val mean: Double
) : Stats

fun stats(list: List<Int>): Stats = StatsImpl(
    list.min(),
    list.max(),
    list.sumOf { it.toLong() },
    list.sumOf { it.toDouble() } / list.size,
)

val stats = stats(listOf(1, 2, 3, 4, 5))

require(stats.min == 1)
require(stats.max == 5)
require(stats.mean == 3.0)
require(stats.sum == 15L)

Did the solution make multiple passes through the list?  If so, we might need a re-think.

It turns out the function now needs to handle a case where the list of numbers may be too big to hold in memory. Instead
of passing the list, we instead pass a function that returns the next element in a (possibly long) series of numbers.

For testing purposes, we need to write small utility function that converts a list into this kind of function. Just for show, we make it an extension function:

In [24]:
@Suppress("unused")
fun List<Int>.provider(): () -> Int? {
    var index = 0
    return {
        takeIf { index < size }?.let { get(index).also { index++ } }
    }
}

val provider = listOf(1,2,3).provider()

require(provider() == 1)
require(provider() == 2)
require(provider() == 3)
require(provider() == null)

Here is a revised edition which can handle abitrarily long "lists", and which relies on our `provider` function for testing purposes:

In [33]:
import java.math.BigInteger

interface Stats {
    val min: Int
    val max: Int
    val sum: BigInteger
    val mean: Double
}

data class StatsImpl(
    override val min: Int,
    override val max: Int,
    override val sum: BigInteger,
    val count: Int
) : Stats {

    constructor() : this(Int.MAX_VALUE, Int.MIN_VALUE, BigInteger.ZERO, 0)

    override val mean: Double
        get() = count.takeIf { it > 0 }
            ?.let { (sum.div(it.toBigInteger())).toDouble() }
            ?: 0.toDouble()

    fun plus(i: Int) = StatsImpl(
        min(this.min, i),
        max(this.max, i),
        this.sum + i.toBigInteger(),
        this.count + 1
    )
}

fun stats(list: () -> Int?): Stats {
    return generateSequence(list).fold(
        StatsImpl(),
        StatsImpl::plus
    )
}

fun List<Int>.provider(): () -> Int? {
    var index = 0
    return {
        takeIf { index < size }?.let { get(index).also { index++ } }
    }
}

val provider = listOf(1, 2, 3, 4, 5).provider()

val stats = stats(provider)

require(stats.min == 1)
require(stats.max == 5)
require(stats.mean == 3.0)
require(stats.sum == 15.toBigInteger())

In another bizarre twist of fate, it now turns out that our list traversal logic is noticed by our colleagues.  They see it as a stable and mature infrastructure, and they want to "hook into" our code with completely arbitary processing logic.  Make it generic!

The below example is a simple "sum" function which hooks into our wonderful machine, generically named `process`:

In [30]:
fun <S> process(list: () -> Int?, initial: () -> S, processor: (S, Int) -> S): S =
    generateSequence(list).fold(initial(), processor)

fun List<Int>.provider(): () -> Int? {
    var index = 0
    return {
        takeIf { index < size }?.let { get(index).also { index++ } }
    }
}

val initial = { 0 }
val processor: (Int, Int) -> Int = { i1, i2 -> i1 + i2 }
val provider = listOf(1, 2, 3, 4, 5).provider()

require(process(provider, initial, processor) == 15)
